In [43]:
import pandas as pd
import os

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Create a directory to store the text files
output_directory = "output_text_files"
os.makedirs(output_directory, exist_ok=True)


In [46]:
excel_file='/content/drive/MyDrive/Input.xlsx'
data=pd.read_excel(excel_file)

In [47]:
data.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [48]:
#libratries for web scrapping
import requests
from bs4 import BeautifulSoup

In [49]:
data.dtypes

URL_ID    float64
URL        object
dtype: object

In [50]:
data['URL_ID'].nunique

<bound method IndexOpsMixin.nunique of 0        123.0
1        321.0
2       2345.0
3       4321.0
4        432.0
        ...   
109    50921.0
110    51382.8
111    51844.6
112    52306.4
113    52768.2
Name: URL_ID, Length: 114, dtype: float64>

In [51]:

headers ={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}

In [52]:
c=0
for url_id, url in zip(data['URL_ID'], data['URL']):
    try:
        r = requests.get(url)
        r.raise_for_status()  # Check for HTTP errors

        soup = BeautifulSoup(r.content, 'lxml')
        the_title = soup.title.text

        # Try finding content in the first class
        content = soup.find(class_="td-post-content tagdiv-type")

        if content is None:
            # If not found, try finding content in the second class
            content = soup.find(class_="tdb-block-inner td-fix-index")

        if content:
            content_text = content.text
            text_file_name = os.path.join(output_directory, f"{url_id}.txt")

            with open(text_file_name, "w", encoding="utf-8") as file:
                file.write(the_title + "\n")
                file.write(content_text)

            print(f"Title and content for URL_ID '{url_id}' have been saved to '{text_file_name}'")
            c=c+1
        else:
            print(f"No content found for URL_ID '{url_id}'")

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve URL for URL_ID '{url_id}': {e}")
    except Exception as e:
        print(f"An error occurred for URL_ID '{url_id}': {e}")
print(c)


Title and content for URL_ID '123.0' have been saved to 'output_text_files/123.0.txt'
Title and content for URL_ID '321.0' have been saved to 'output_text_files/321.0.txt'
Title and content for URL_ID '2345.0' have been saved to 'output_text_files/2345.0.txt'
Title and content for URL_ID '4321.0' have been saved to 'output_text_files/4321.0.txt'
Title and content for URL_ID '432.0' have been saved to 'output_text_files/432.0.txt'
Title and content for URL_ID '2893.8' have been saved to 'output_text_files/2893.8.txt'
Title and content for URL_ID '3355.6' have been saved to 'output_text_files/3355.6.txt'
Title and content for URL_ID '3817.4' have been saved to 'output_text_files/3817.4.txt'
Title and content for URL_ID '4279.2' have been saved to 'output_text_files/4279.2.txt'
Title and content for URL_ID '4741.0' have been saved to 'output_text_files/4741.0.txt'
Title and content for URL_ID '5202.8' have been saved to 'output_text_files/5202.8.txt'
Title and content for URL_ID '5664.6' 

In [53]:
pip install nltk

In [54]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
import string
# Define a translation table to remove punctuation
translator = str.maketrans('', '', string.punctuation + "–")

In [56]:
# Initialize an empty dictionary to store tokenized text for each file (keyed by filename)
tokenized_texts = {}
token_sentence ={}
personal_pronoun={}


tokenize the text

In [57]:
files = os.listdir(output_directory)
for thefile in files:
  if thefile.endswith(".txt"):
    file_path = os.path.join(output_directory,thefile)
    with open(file_path, 'r') as file:
      content=file.read().lower().translate(translator)
    #store personal pronoun
    #personal_pronoun[thefile]=count_personal_pronouns(content)
    #Tokenize the text into words
    if thefile not in tokenized_texts:
      tokenized_texts[thefile]= word_tokenize(content)







In [58]:
len(tokenized_texts)

112

sentece tokenize

In [59]:
for thefile in files:
  if thefile.endswith(".txt"):
    file_path = os.path.join(output_directory,thefile)
    with open(file_path, 'r') as file:
      if thefile not in token_sentence:
        token_sentence[thefile]=nltk.sent_tokenize(file.read())



Sentimental analysis
1.1  

In [60]:
combined_text = ''

In [61]:
#define the stop words
stopwords = []


# List all files in the directory
file_names = os.listdir('/content/drive/MyDrive/StopWords')

for file_name in file_names:
    if file_name.endswith('.txt'):
        file_path = os.path.join('/content/drive/MyDrive/StopWords',file_name)

        # Read and process the content of each text file
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            file_content = file.read()
            combined_text += file_content




In [62]:
combined_text = combined_text.lower()

# Remove punctuation from the combined text
translator = str.maketrans('', '', string.punctuation)
cleaned_stopwords = combined_text.translate(translator)

# Tokenize the cleaned text into words
stopword_tokens = word_tokenize(cleaned_stopwords)

In [63]:
# Store the tokens in the stopwords list
stopwords.extend(stopword_tokens)

In [64]:
len(stopwords)

14236

Data cleaning

In [65]:
#filter tokens by removing words that are in the stopwords list
for thefile in files:
  if thefile.endswith(".txt"):
    tokenized_texts[thefile]= [word for word in tokenized_texts[thefile] if word not in stopwords]



defining the positive and negative words dictionary

In [66]:
#filling the positive words list
positivefile_path = '/content/drive/MyDrive/positive-words.txt'
negativefile_path = '/content/drive/MyDrive/negative-words.txt'

with open(positivefile_path, 'r', encoding='utf-8') as file:
  words= file.read().lower().translate(translator)

# Tokenize the cleaned text into words
positivewords= word_tokenize(words)


#filling the negative words list
#############################################################################################################################

with open(negativefile_path, 'r', encoding='latin-1') as file:
  words1= file.read().lower().translate(translator)

# Tokenize the cleaned text into words
negativewords= word_tokenize(words1)









In [67]:
pip install syllables

evaluating the scores

In [68]:
positive_score={}
negative_score={}
word_count={}
polarity_score={}
subjectivity_score={}
Average_Sentence_Len={}
complex_count={}
percent_complexwords={}
Fog_Index={}
syllable_count={}
personal_pronoun={}


In [69]:
import re

In [70]:
def count_complex_words(words_list):
  c = 0
  for word in words_list:
    l = re.findall('(?!e$)[aeiou]+', word, re.I)+re.findall('^[aeiouy]*e$', word, re.I)
    if len(l) > 2:
      c += 1
  return c

In [71]:
def count_syllables(word):
  c = 0
  vowels = 'aeiou'
  l = re.findall(f'(?!e$)(?!es$)(?!ed$)[{vowels}]', word, re.I)
  return len(l)

In [72]:
for thefile in files:
    if thefile.endswith(".txt"):
        syllable_per_word = []
        for word in tokenized_texts[thefile]:
            list1 = []
            list1.append(word)
            list1.append(count_syllables(word))
            syllable_per_word.append(list1)

        syllable_sum = 0
        for i in range(len(syllable_per_word)):
            syllable_sum += syllable_per_word[i][1]

        syllable_count[thefile] = int(syllable_sum / len(syllable_per_word))


In [73]:
def count_personal_pronouns(text):
  pronoun_count = re.compile(r'\b(I|we|ours|my|mine|(?-i:us))\b', re.I)
  pronouns = pronoun_count.findall(text)
  return len(pronouns)

In [74]:
main_dict={}
for thefile in files:
  if thefile.endswith(".txt"):
    positive_score[thefile] = sum(1 for word in tokenized_texts[thefile] if word in positivewords)
    negative_score[thefile] = -sum(-1 for word in tokenized_texts[thefile] if word in negativewords)
    word_count[thefile]= len(tokenized_texts[thefile])
    polarity_score[thefile]= (positive_score[thefile] - negative_score[thefile])/((positive_score[thefile] + negative_score[thefile]) + 0.000001)
    subjectivity_score[thefile]=(positive_score[thefile] + negative_score[thefile]) /(word_count[thefile] + 0.000001)
    Average_Sentence_Len[thefile]=word_count[thefile]/len(token_sentence[thefile])
    complex_count[thefile]=count_complex_words(tokenized_texts[thefile])
    percent_complexwords[thefile]=complex_count[thefile]/word_count[thefile]
    Fog_Index[thefile]= 0.4 * (Average_Sentence_Len[thefile] + percent_complexwords[thefile] )

    file_dict = {
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'WORD COUNT': word_count,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': Average_Sentence_Len,
        'COMPLEX WORD COUNT': complex_count,
        'PERCENT COMPLEX WORDS': percent_complexwords,
        'FOG INDEX': Fog_Index,

         }

     # Add the file_dict to the main_dict using the file name as the key
#main_dict[thefile] = file_dict
  # main_dict.update(positive_score[thefile])
  # main_dict.update(negative_score[thefile])
  # main_dict.update(word_count[thefile])
  # main_dict.update(polarity_score[thefile])
  # main_dict.update(subjectivity_score[thefile])
  # main_dict.update(Average_Sentence_Len[thefile])
  # main_dict.update(complex_count[thefile])
  # main_dict.update(percent_complexwords[thefile])
  # main_dict.update(Fog_Index[thefile])
  # main_dict.update(percent_complexwords[thefile])
  # main_dict.update(personal_pronoun[thefile])





  # main_list=[positive_score[thefile],negative_score[thefile],
  #        word_count[thefile], polarity_score[thefile],
  #          subjectivity_score[thefile], Average_Sentence_Len[thefile],
  #            Average_Sentence_Len[thefile],complex_count[thefile],
  #            percent_complexwords[thefile],Fog_Index[thefile]]








In [75]:
df1 = pd.DataFrame.from_dict(file_dict).reset_index()

In [76]:
df1.reset_index(drop=True, inplace=True)
df1.index.name = None

In [77]:
df1

,index,POSITIVE SCORE,NEGATIVE SCORE,WORD COUNT,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENT COMPLEX WORDS,FOG INDEX
0,40761.4.txt,0,0,6,0.000000,0.000000,3.000000,3,0.500000,1.400000
1,43994.0.txt,44,48,710,-0.043478,0.129577,10.142857,226,0.318310,4.184467
2,21827.6.txt,4,0,85,1.000000,0.047059,7.727273,35,0.411765,3.255615
3,35681.6.txt,17,41,888,-0.413793,0.065315,14.322581,345,0.388514,5.884438
4,23213.0.txt,48,18,614,0.454545,0.107492,10.233333,272,0.442997,4.270532
...,...,...,...,...,...,...,...,...,...,...
107,321.0.txt,38,13,287,0.490196,0.177700,11.480000,135,0.470383,4.780153
108,30140.0.txt,37,33,642,0.057143,0.109034,11.888889,241,0.375389,4.905711
109,51844.6.txt,86,32,894,0.457627,0.131991,12.771429,327,0.365772,5.254880
110,43532.2.txt,22,36,586,-0.241379,0.098976,10.654545,212,0.361775,4.406528


In [78]:
df1['index'] = df1['index'].str.replace('.txt', '', regex=False)


In [79]:
df1['index']


0      40761.4
1      43994.0
2      21827.6
3      35681.6
4      23213.0
        ...   
107      321.0
108    30140.0
109    51844.6
110    43532.2
111    45379.4
Name: index, Length: 112, dtype: object

In [80]:
df1 = df1.rename(columns={'index': 'URL_ID'})

In [81]:
df1.head()

,URL_ID,POSITIVE SCORE,NEGATIVE SCORE,WORD COUNT,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENT COMPLEX WORDS,FOG INDEX
0,40761.4,0,0,6,0.000000,0.000000,3.000000,3,0.500000,1.400000
1,43994.0,44,48,710,-0.043478,0.129577,10.142857,226,0.318310,4.184467
2,21827.6,4,0,85,1.000000,0.047059,7.727273,35,0.411765,3.255615
3,35681.6,17,41,888,-0.413793,0.065315,14.322581,345,0.388514,5.884438
4,23213.0,48,18,614,0.454545,0.107492,10.233333,272,0.442997,4.270532


In [82]:
data['URL_ID']

0        123.0
1        321.0
2       2345.0
3       4321.0
4        432.0
        ...   
109    50921.0
110    51382.8
111    51844.6
112    52306.4
113    52768.2
Name: URL_ID, Length: 114, dtype: float64

In [83]:
df1['URL_ID'] = df1['URL_ID'].astype(float)
result_df = pd.merge(df1,data, on='URL_ID')

In [84]:
result_df

,URL_ID,POSITIVE SCORE,NEGATIVE SCORE,WORD COUNT,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENT COMPLEX WORDS,FOG INDEX,URL
0,40761.4,0,0,6,0.000000,0.000000,3.000000,3,0.500000,1.400000,https://insights.blackcoffer.com/how-covid-19-...
1,43994.0,44,48,710,-0.043478,0.129577,10.142857,226,0.318310,4.184467,https://insights.blackcoffer.com/estimating-th...
2,21827.6,4,0,85,1.000000,0.047059,7.727273,35,0.411765,3.255615,https://insights.blackcoffer.com/how-we-foreca...
3,35681.6,17,41,888,-0.413793,0.065315,14.322581,345,0.388514,5.884438,https://insights.blackcoffer.com/coronavirus-i...
4,23213.0,48,18,614,0.454545,0.107492,10.233333,272,0.442997,4.270532,https://insights.blackcoffer.com/management-ch...
...,...,...,...,...,...,...,...,...,...,...,...
107,321.0,38,13,287,0.490196,0.177700,11.480000,135,0.470383,4.780153,https://insights.blackcoffer.com/rise-of-e-hea...
108,30140.0,37,33,642,0.057143,0.109034,11.888889,241,0.375389,4.905711,https://insights.blackcoffer.com/how-small-bus...
109,51844.6,86,32,894,0.457627,0.131991,12.771429,327,0.365772,5.254880,https://insights.blackcoffer.com/what-are-the-...
110,43532.2,22,36,586,-0.241379,0.098976,10.654545,212,0.361775,4.406528,https://insights.blackcoffer.com/coronavirus-i...


In [86]:
result_df.to_csv('output_file.csv', index=False)